In [1]:
import torch
import torch.nn.functional as F

class ConvTranspose2dCustom:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        
        # Initialize the kernel (weights) and bias
        self.weight = torch.randn(out_channels, in_channels, kernel_size, kernel_size)
        self.bias = torch.randn(out_channels)
    
    def forward(self, x):
        # 1. Get the input dimensions
        batch_size, in_channels, in_height, in_width = x.size()
        
        # 2. Calculate output dimensions
        out_height = (in_height - 1) * self.stride - 2 * self.padding + self.kernel_size
        out_width = (in_width - 1) * self.stride - 2 * self.padding + self.kernel_size
        
        # 3. Initialize the output tensor
        output = torch.zeros(batch_size, self.out_channels, out_height, out_width)
        
        # 4. Upsample the input by inserting zeros based on stride
        upsampled_x = torch.zeros(batch_size, in_channels, in_height * self.stride, in_width * self.stride)
        upsampled_x[:, :, ::self.stride, ::self.stride] = x
        
        # 5. Perform the convolution operation manually
        for i in range(out_height):
            for j in range(out_width):
                h_start = i - self.padding
                w_start = j - self.padding
                h_end = h_start + self.kernel_size
                w_end = w_start + self.kernel_size
                
                if h_start >= 0 and w_start >= 0 and h_end <= upsampled_x.shape[2] and w_end <= upsampled_x.shape[3]:
                    region = upsampled_x[:, :, h_start:h_end, w_start:w_end]
                    output[:, :, i, j] = (region * self.weight).sum(dim=[1, 2, 3]) + self.bias

        return output

# Example usage
input_tensor = torch.tensor([[[[1., 2., 3., 4.],
                               [5., 6., 7., 8.],
                               [9., 10., 11., 12.],
                               [13., 14., 15., 16.]]]])

# Initialize custom ConvTranspose2d
deconv_custom = ConvTranspose2dCustom(in_channels=1, out_channels=1, kernel_size=3, stride=2, padding=1)

# Perform forward pass
output = deconv_custom.forward(input_tensor)

# Output shape
print("Custom Transposed Convolution output shape:", output.shape)
print("Custom Transposed Convolution output:", output)


Custom Transposed Convolution output shape: torch.Size([1, 1, 7, 7])
Custom Transposed Convolution output: tensor([[[[  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
          [  0.0000,  -0.7829,   2.4113,  -0.1416,   2.2920,   0.4998,   2.1727],
          [  0.0000,  -3.2963,  -5.3242,  -3.7741,  -6.1874,  -4.2520,  -7.0506],
          [  0.0000,   1.7825,   1.9340,   2.4238,   1.8147,   3.0652,   1.6954],
          [  0.0000,  -5.2078,  -8.7769,  -5.6857,  -9.6401,  -6.1635, -10.5033],
          [  0.0000,   4.3479,   1.4568,   4.9892,   1.3375,   5.6305,   1.2182],
          [  0.0000,  -7.1193, -12.2297,  -7.5972, -13.0929,  -8.0751, -13.9561]]]])


In [4]:
import torch
import torch.nn.functional as F
import torch.nn as nn

class TransposedConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0):
        super(TransposedConv2d, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.output_padding = output_padding
        
        # Define a regular Conv2d layer
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, bias=False)
        
        # Reverse the kernel to simulate transposed convolution
        self.conv.weight.data = torch.flip(self.conv.weight.data, [2, 3])

    def forward(self, x):
        # Step 1: Expand the input by inserting zeros (simulate stride > 1)
        batch_size, channels, height, width = x.size()
        expanded_height = (height - 1) * self.stride + 1
        expanded_width = (width - 1) * self.stride + 1

        expanded_x = torch.zeros(batch_size, channels, expanded_height, expanded_width, device=x.device)
        expanded_x[:, :, ::self.stride, ::self.stride] = x

        # Step 2: Apply the flipped convolution (simulating transposed convolution)
        output = F.conv2d(expanded_x, self.conv.weight, bias=None, stride=1, padding=self.padding)

        # Step 3: Adjust the output size by considering output padding
        if self.output_padding > 0:
            output = F.pad(output, (0, self.output_padding, 0, self.output_padding))

        return output

# Example Usage:
input_tensor = torch.tensor([[[[1., 2.], [3., 4.]]]])  # 1x1x2x2 input
model = TransposedConv2d(in_channels=1, out_channels=1, kernel_size=2, stride=4, padding=0)
output = model(input_tensor)

print("Output:\n", output)


Output:
 tensor([[[[-0.1828,  0.0000,  0.0000, -0.8982],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [-1.3064,  0.0000,  0.0000, -0.1076]]]],
       grad_fn=<ConvolutionBackward0>)
